In [ ]:
from hashlib import sha1
import hmac
import binascii
def getUrl(request):
    devId = 3002228
    key = '5bef2099-6493-48e4-b344-e37e99530ce6'
    request = request + ('&' if ('?' in request) else '?')
    raw = request+'devid={0}'.format(devId)
    #print(raw.encode())
    key_bin = key.encode()
    raw_bin = raw.encode() 
    hashed = hmac.new(key_bin, raw_bin, sha1)
    signature = hashed.hexdigest()
    return 'http://timetableapi.ptv.vic.gov.au'+raw+'&signature={1}'.format(devId, signature)
#print(getUrl('/v3/route_types'))


import requests
import json
def download_get(url_name,file_path):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    with open(file_path,'w') as f:
        f.write(response.text)
def download_get_to_dict(url_name):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    return json.loads(response.text)

#download_get('/v3/route_types','/Users/chunhua/tmp_share/heyonglin/route_types.json')
#download_get('/v3/routes','/Users/heyonglin/routes.json')

routes_json_file = '/Users/heyonglin/route/routes.json'
# routes_json_file = '../data/routes.json'
with open(routes_json_file) as f:
    route_dict = json.load(f)
    # print(route_dict.keys())
    stops_files = [open(f'/Users/heyonglin/route/stops_file_{i}.json','a')  for i in range(5)]
    for stop_file in stops_files:
        stop_file.write('[\n')
    is_first_line = [True,True,True,True,True]
    for route_item in route_dict['routes']:
        route_id = route_item['route_id']
        route_type = route_item['route_type']
        url_name = f'/v3/stops/route/{route_id}/route_type/{route_type}'
        res = download_get_to_dict(url_name)
        stops = res['stops']
        for stop in stops:
            if not is_first_line[route_type]:
                stops_files[route_type].write(',\n')
            json.dump(stop,stops_files[route_type])
            is_first_line[route_type] = False
    
    for stop_file in stops_files:
        stop_file.write('\n]')
        stop_file.close()    